<a href="https://colab.research.google.com/github/todnewman/coe_training/blob/master/MNIST_Classifier_keras+tensorboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generic Convolutional Neural Network example with Tensorboard Visualizations

This example is intended to provide a very simple solution to MNIST classification using Keras (a convenient front-end for Tensorflow).  Additionally, it demonstrates how to connect to Tensorboard through CoLaboratory.

MNIST is the hand-written digits dataset created by NIST and later modified to fit in a 28x28 pixel bounding box and anti-aliased to include grayscale.  It has been used for years to evaluate Image processing and machine learning algorithms.  The CNN is the current champion, with error rates as low as 0.2%.

Author: W. Tod Newman
Learning Objectives:
* Learn how to set up a simple Tensorflow model 
* Learn how to simplify the Tensorflow model definition using Keras
* Understand how to connect to Tensorboard and how to evaluate metrics in Tensorboard
* Understand how to use this example with a custom image dataset


## Tensorboard Overview
The computations you'll use TensorFlow for - like training a massive deep neural network - can be complex and confusing. To make it easier to understand, debug, and optimize TensorFlow programs, Google has included a suite of visualization tools called TensorBoard. You can use TensorBoard to visualize your TensorFlow graph, plot quantitative metrics about the execution of your graph, and show additional data like images that pass through it.

Here's a good overview of what you can do with Tensorboard.  I'd watch the video on this link first.  https://www.tensorflow.org/guide/summaries_and_tensorboard


# First, we need to connect to Tensorboard from CoLaboratory
As you probably know, your Google Colab virtual machine is running on a local network located in a Google's server room, while your local machine could be anywhere else in the world.

How then can we access the TensorBoard page from our local machine?

Fortunately there's a free service named ngrok that will allow us to connect to the Tensorboard connection on our Web browser.

## Step One: Get the ngrok image from its stable location, then unzip it. 
Remember, this is all being done in the Google environment, so don't worry about your firewall...
ngrok is pretty useful for testing.  Find more at https://ngrok.com/

In [6]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package ngrok


## Step Two: Run TensorBoard

In [0]:
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

## Step Three: Run ngrok

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

## Step Four:  Grab the URL that the Tensorboard instance will be hosted at.
The URL below (in blue) is what you click on to see your Tensorboard instance after you train the CNN.

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://2bee512a.ngrok.io


## MNIST Classifer in Keras with Tensorboard metrics

This is a standard MNIST classifier.  It classifies over 99% at 12 epochs (an epoch is a complete pass through the training data), and that's even without tweaking hyperparameters.  After running this, click on the Tensorboard link in the block above this (something like http://***.ngrok.io)

In [0]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import TensorBoard

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions (in this case, MNIST images)
img_rows, img_cols = 28, 28

# First, we load the mnist data.  This is a built-in call, as MNIST is such
# a common dataset to use to test image classifiers.  Then we will take
# the data, shuffle it and split it between train and test sets

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Evaluate which backend data format we have.  Tensorflow has its own backend,
# but some people use Theano or other backends that have different ways of 
# structuring the image data. This has to do with efficiency on CPUs/GPUs.

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
# Here we cast the image data to a 32 bit float and normalize.  This improves
# neural net performance.
# INFO: The color components of an 8-bit RGB image are integers in the 
# range [0, 255] rather than floating-point values in the range [0, 1]. 
# A pixel whose color components are (255,255,255) is displayed as white. 
# To take this to a normalized floating point value, therefore, we divide 
# by 255.

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Will print out shape of the data in NHWC format, where N refers to the number 
# of images in a batch, H refers to the number of pixels in the vertical 
# dimension, W refers to the number of pixels in the horizontal dimension, 
# and C refers to the channels (e.g. 1 for black and white, 3 for RGB, etc.)

print('x_train shape in NHWC format:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Define our model below.  We have two 2D convolution layers [32,64]
# followed by pooling (we do this to control overfitting) and a 128 neuron
# fully-connected layer.  We're using dropout here to randomly reduce the 
# network complexity.  The classification layer is a Softmax.  
# The Adadelta optimizer adapts learning rates based on a moving window of 
# gradient updates, which has the effect of extending learning longer.

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# Call TensorBoard for visual metrics

tbCallBack = TensorBoard(log_dir='./log', histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=batch_size,
                         write_images=True)

# Fit the model based on everything we've defined to date.  Write metrics to 
# TensorBoard

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[tbCallBack])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Using TensorFlow backend.


11493376/11490434 [==============================] - 0s 0us/step
x_train shape in NHWC format: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 12s 203us/step - loss: 0.2679 - acc: 0.9176 - val_loss: 0.0600 - val_acc: 0.9811
Epoch 2/12
60000/60000 [==============================] - 9s 157us/step - loss: 0.0863 - acc: 0.9746 - val_loss: 0.0392 - val_acc: 0.9868
Epoch 3/12
60000/60000 [==============================] - 9s 157us/step - loss: 0.0655 - acc: 0.9804 - val_loss: 0.0322 - val_acc: 0.9885
Epoch 4/12
60000/60000 [==============================] - 9s 158us/step - loss: 0.0520 - acc: 0.9844 - val_loss: 0.0356 - val_acc: 0.9879
Epoch 5/12
60000/60000 [==============================] - 9s 157us/step - loss: 0.0450 - acc: 0.9865 - val_loss: 0.0286 - val_acc: 0.9913
Epoch 6/12
60000/60000 [==============================] - 9s 157us/step - loss: 0.0404 - acc: 0.9873 - val